# II. From Scratch

### 1. Import libs

In [3]:
import math
import os
import re
import time
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

/home/trongnhat/anaconda3/envs/poem/lib/python3.9/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/trongnhat/anaconda3/envs/poem/lib/python3.9/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/trongnhat/anaconda3/envs/poem/lib/python3.9/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/

### 2. Datasets

In [3]:
DATASET_PATH = 'poem_dataset_final/poem_final.csv'
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [4]:
df['content'][0].split('\n')

['Cái làm ta hạnh phúc',
 'Thực ra cũng chẳng nhiều',
 'Chỉ cần có ai đó',
 'Để ta thầm thương yêu',
 '',
 'Rồi thêm chút công việc',
 'Cho ta làm hàng ngày',
 'Cuối cùng, chút mơ mộng',
 'Để đưa ta lên mây']

### 3. Build vectorization function

In [5]:
def text_normalize(text): # khong can xoa di dau cau hay doi chu hoa thanh chu thuong
    text = text.strip() # Xoa khoang trang dau cuoi

    return text

df['content'] = df['content'].apply(lambda x: text_normalize(x))

In [6]:
def tokenizer(text):
    return text.split()

def yield_tokens(df):
    for idx, row in df.iterrows():
        yield tokenizer(row['content'])

vocab = build_vocab_from_iterator(
    yield_tokens(df),
    specials=['<unk>', '<pad>', '<sos>', '<eos>', '<eol>']
)
vocab.set_default_index(vocab['<unk>'])
vocab.get_stoi()

{'“Liệu': 2198,
 'ẩm': 2196,
 'ầm': 2195,
 'đợi,': 2193,
 'địa': 2187,
 'đền!': 2186,
 'đếm': 2185,
 'đặn': 2184,
 'đáng': 2182,
 'điên': 2178,
 'Được': 2174,
 'Đâu': 2172,
 'Đàn': 2171,
 'ô-kê': 2169,
 'Ðức': 2164,
 'Ðêm': 2162,
 'Ðã': 2161,
 'xấu': 2158,
 'xúc': 2156,
 'xoãi': 2154,
 'vụng': 2151,
 'về…': 2150,
 'vạng': 2148,
 'tột': 2144,
 'tổng': 2143,
 'tất': 2141,
 'túi': 2138,
 'tôi.': 2137,
 'tình.': 2134,
 'trưởng': 2132,
 'trách': 2129,
 'trung,': 2128,
 'trao': 2127,
 'toát': 2126,
 'thỏ': 2121,
 'thong': 2114,
 'xiêu': 2153,
 'thiểu': 2112,
 'sướng': 2110,
 'rợ.': 2104,
 'rộng': 2103,
 'rẩy': 2102,
 'rèn': 2100,
 'riết': 2096,
 'phứt': 2093,
 'phú': 2091,
 'phù.': 2090,
 'phím': 2089,
 'nứt': 2083,
 'nợ': 2081,
 'thon': 2113,
 'nề': 2079,
 'nước...': 2078,
 'vang': 2145,
 'nông': 2076,
 'nó,': 2075,
 'nâu': 2074,
 'nuốt': 2073,
 'nhằn': 2070,
 'nhạt': 2067,
 'nhưng': 2066,
 'nhung': 2065,
 'nhiệm': 2063,
 'ngó': 2058,
 'nghị': 2056,
 'mục.': 2054,
 'mổ': 2053,
 'mệt': 2051,

In [7]:
PAD_TOKEN = vocab['<pad>']
EOS_TOKEN = vocab['<eos>']

MAX_SEQ_LEN = 25

def pad_and_truncate(input_ids, max_seq_len):
    if len(input_ids) > max_seq_len:
        input_ids = input_ids[:max_seq_len]
    else:
        input_ids += [PAD_TOKEN] * (max_seq_len - len(input_ids))

    return input_ids

def vectorize(text, max_seq_len):
    input_ids = [vocab[token] for token in tokenizer(text)]
    input_ids = pad_and_truncate(input_ids, max_seq_len)

    return input_ids

def decode(input_ids):
    return [vocab.get_itos()[token_id] for token_id in input_ids]

In [8]:
vocab.get_itos()[0]

'<unk>'

In [9]:
print(df['content'][0].split('\n')[0])
print(vectorize(df['content'][0].split('\n')[0], 10))

Cái làm ta hạnh phúc
[175, 62, 39, 313, 366, 1, 1, 1, 1, 1]


### 3. Create Poem Dataset

In [10]:
class PoemDataset(Dataset):
    def __init__(self, df, tokenizer, vectorizer, max_seq_len):
        self.tokenizer = tokenizer
        self.vectorizer = vectorizer
        self.max_seq_len = max_seq_len
        self.input_seqs, self.target_seqs, self.padding_masks = self.create_samples(df)

    def create_padding_mask(self, input_ids, pad_token_id=PAD_TOKEN):
        return [0 if token_id == pad_token_id else 1 for token_id in input_ids]

    def split_content(self, content):
        samples = []

        poem_parts = content.split('\n\n')
        for poem_part in poem_parts:
            poem_in_lines = poem_part.split('\n')
            if len(poem_in_lines) == 4:
                samples.append(poem_in_lines)

        return samples

    def prepare_sample(self, sample):
        input_seqs = []
        target_seqs = []
        padding_masks = []

        input_text = '<sos> ' + ' <eol> '.join(sample) + ' <eol> <eos>'
        input_ids = self.tokenizer(input_text)
        for idx in range(1, len(input_ids)):
            input_seq = ' '.join(input_ids[:idx])
            target_seq = ' '.join(input_ids[1:idx+1])
            input_seq = self.vectorizer(input_seq, self.max_seq_len)
            target_seq = self.vectorizer(target_seq, self.max_seq_len)
            padding_mask = self.create_padding_mask(input_seq)

            input_seqs.append(input_seq)
            target_seqs.append(target_seq)
            padding_masks.append(padding_mask)

        return input_seqs, target_seqs, padding_masks

    def create_samples(self, df):
        input_seqs = []
        target_words = []
        padding_masks = []

        for idx, row in df.iterrows():
            content = row['content']
            samples = self.split_content(content)
            for sample in samples:
                sample_input_seqs, sample_target_words, sample_padding_masks = self.prepare_sample(sample)

                input_seqs += sample_input_seqs
                target_words += sample_target_words
                padding_masks += sample_padding_masks

        input_seqs = torch.tensor(input_seqs, dtype=torch.long)
        target_words = torch.tensor(target_words, dtype=torch.long)
        padding_masks = torch.tensor(padding_masks, dtype=torch.float)

        return input_seqs, target_words, padding_masks

    def __len__(self):
        return len(self.input_seqs)

    def __getitem__(self, idx):
        input_seqs = self.input_seqs[idx]
        target_seqs = self.target_seqs[idx]
        padding_masks = self.padding_masks[idx]

        return input_seqs, target_seqs, padding_masks

TRAIN_BS = 256
train_dataset = PoemDataset(
    df=df,
    tokenizer=tokenizer,
    vectorizer=vectorize,
    max_seq_len=MAX_SEQ_LEN
)
train_loader = DataLoader(
    train_dataset,
    batch_size=TRAIN_BS,
    shuffle=False
)

In [11]:
input_seqs, target_seqs, padding_masks = next(iter(train_loader))

print(input_seqs[0])
print(target_seqs[0])
print(padding_masks[0])

tensor([2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1])
tensor([175,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1])
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])


In [12]:
for idx in range(MAX_SEQ_LEN):
    print(decode(input_seqs[idx]))
    print(decode(target_seqs[idx]))

['<sos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['Cái', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<sos>', 'Cái', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['Cái', 'làm', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
['<sos>', 'Cái', 'làm', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',

### Create Model

In [23]:
class PositionalEncoding(nn.Module):
    def __init__(self, embedding_dims, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dims, 2) * (-math.log(10000.0) / embedding_dims))
        pe = torch.zeros(max_len, 1, embedding_dims)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        x = self.dropout(x)

        return x

class TransformerModel(nn.Module):
    def __init__(
        self,
        vocab_size,
        embedding_dims,
        n_heads,
        hidden_dims,
        n_layers,
        dropout=0.5
    ):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.embedding = nn.Embedding(vocab_size, embedding_dims)
        self.embedding_dims = embedding_dims

        self.pos_encoder = PositionalEncoding(embedding_dims, dropout)
        encoder_layers = nn.TransformerEncoderLayer(
            embedding_dims,
            n_heads,
            hidden_dims,
            dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)
        self.linear = nn.Linear(embedding_dims, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask=None, padding_mask=None):
        src = self.embedding(src) * math.sqrt(self.embedding_dims)
        src = self.pos_encoder(src)
        if src_mask is None:
            src_mask = nn.Transformer.generate_square_subsequent_mask(len(src)).to(device)
        output = self.transformer_encoder(src, mask=src_mask, src_key_padding_mask=padding_mask)
        output = self.linear(output)

        return output

In [24]:
VOCAB_SIZE = len(vocab)
EMBEDDING_DIMS = 128
HIDDEN_DIMS = 128
N_LAYERS = 2
N_HEADS = 4
DROPOUT = 0.2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_tests = torch.randint(1, 10, (1, 10)).to(device)

model = TransformerModel(
    VOCAB_SIZE,
    EMBEDDING_DIMS,
    N_HEADS,
    HIDDEN_DIMS,
    N_LAYERS,
    DROPOUT
).to(device)

with torch.no_grad():
    output = model(input_tests)
    print(output.shape)

torch.Size([1, 10, 2201])


/home/trongnhat/anaconda3/envs/poem/lib/python3.9/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


### Training

In [15]:
LR = 5.0
EPOCHS = 100

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

In [17]:
model.train()
for epoch in range(EPOCHS):
    losses = []
    for idx, samples in enumerate(train_loader):
        input_seqs, target_seqs, padding_masks = samples
        input_seqs = input_seqs.to(device)
        target_seqs = target_seqs.to(device)
        padding_masks = padding_masks.to(device).permute(1, 0)

        output = model(input_seqs, padding_mask=padding_masks)
        output = output.permute(0, 2, 1)
        loss = criterion(output, target_seqs)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        losses.append(loss.item())

    total_loss = sum(losses) / len(losses)
    print(f'EPOCH {epoch+1}\tLoss {total_loss}')
    scheduler.step()

EPOCH 1	Loss 4.839075814593922
EPOCH 2	Loss 3.579755490476435
EPOCH 3	Loss 3.3613596070896494
EPOCH 4	Loss 3.2712796980684455
EPOCH 5	Loss 3.0483466386795044
EPOCH 6	Loss 2.6412847096269783
EPOCH 7	Loss 2.2033758705312554
EPOCH 8	Loss 1.7600067027590491
EPOCH 9	Loss 1.4426114179871299
EPOCH 10	Loss 1.2348678518425336
EPOCH 11	Loss 1.0588045919483358
EPOCH 12	Loss 0.9382781034166162
EPOCH 13	Loss 0.8609840422868729
EPOCH 14	Loss 0.7876197831197218
EPOCH 15	Loss 0.7453668754209172
EPOCH 16	Loss 0.6968556093898687
EPOCH 17	Loss 0.6889602690935135
EPOCH 18	Loss 0.6590083329515024
EPOCH 19	Loss 0.6362459524111315
EPOCH 20	Loss 0.6217625899748369
EPOCH 21	Loss 0.6106516759503972
EPOCH 22	Loss 0.5937107665972277
EPOCH 23	Loss 0.5769970626993612
EPOCH 24	Loss 0.5664967135949568
EPOCH 25	Loss 0.5614972128109499
EPOCH 26	Loss 0.5540683506564661
EPOCH 27	Loss 0.5424841730432077
EPOCH 28	Loss 0.5378327789631757
EPOCH 29	Loss 0.5270772779529745
EPOCH 30	Loss 0.5187155441804365
EPOCH 31	Loss 0.51219

In [25]:
#save model 
# torch.save(model.state_dict(), 'model_poem_gen.pth')

#load model
model.load_state_dict(torch.load('model_poem_gen.pth'))

<All keys matched successfully>

### 6. Inference

In [21]:
def sample_with_temperature(logits, temperature=1.0):
    if temperature != 1.0:
        logits = logits / temperature

    probabilities = F.softmax(logits, dim=-1)

    sampled_index = torch.multinomial(probabilities, 1).item()

    return sampled_index

In [27]:
model.eval()
temperature = 1.2
input_text = '<sos> Anh'
input_tokens = tokenizer(input_text)
input_ids = [vocab[token] for token in input_tokens]
eos_token_id = vocab['<eos>']
generated_ids = input_ids.copy()
MAX_GENERATION_LEN = 50
for _ in range(MAX_GENERATION_LEN):
    input_tensor = torch.tensor([generated_ids], dtype=torch.long).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)

    last_token_logits = outputs[0, -1, :]
    next_token_id = sample_with_temperature(last_token_logits, temperature)
    generated_ids.append(next_token_id)

    if next_token_id == eos_token_id:
        break

# Convert the generated tokens back to text
generated_text = decode(generated_ids)
generated_text = ' '.join(generated_text)
generated_text = generated_text.replace('<sos>', '')
lines = generated_text.split('<eol>')
for line in lines:
    print(''.join(line))

 Anh đi đón mùa tầm xuân 
 Người ấy đến rất say rượu 
 Cành dựa vào hang núi 
 từ đâu bước lại 
 Nhe nanh dài 
 Anh đi qua đời em được trẻ 
 Khi hoá làm vách núi 
 Không có anh đã lịm khói 
 Em


# III. GPT2

### 1. Import Libs

In [26]:
%pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.35.2
%pip install -qq accelerate==0.26.1
!apt install git-lfs

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [13]:
import os
import math
import torch
import pandas as pd

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset

/home/trongnhat/anaconda3/envs/poem/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/trongnhat/anaconda3/envs/poem/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/trongnhat/anaconda3/envs/poem/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [32]:
# notebook_login()

### 2. Datasets

In [4]:
DATASET_PATH = 'poem_dataset_final/poem_final.csv'
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [5]:
print(df['content'][0])

Cái làm ta hạnh phúc
Thực ra cũng chẳng nhiều
Chỉ cần có ai đó
Để ta thầm thương yêu

Rồi thêm chút công việc
Cho ta làm hàng ngày
Cuối cùng, chút mơ mộng
Để đưa ta lên mây


In [6]:
def split_content(content):
    samples = []

    poem_parts = content.split('\n\n')
    for poem_part in poem_parts:
        poem_in_lines = poem_part.split('\n')
        if len(poem_in_lines) == 4:
            samples.append(poem_in_lines)

    return samples

df['content'] = df['content'].apply(lambda x: split_content(x))
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,"[[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhi...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,"[[Chiều vừa xốp trên tay, Chợt nghe thoáng ong...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,"[[Dưới giàn hoa thiên lý, Một mình anh đang ng...","Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","[[Đến, nhiều nơi để đến, Về, trở lại với mình,...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,"[[Đừng bao giờ dại dột, Đem chuyện riêng của m...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"[[Ôi, con sóng chết khô,, vật vờ trong bùn quá...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,"[[Áng dương không biết sầu, Nằm mãi ở trên cao...","Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,[],19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,"[[Không có anh để già, Làm sao em được trẻ, Mu...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [7]:
df_exploded = df.explode('content')
df_exploded.reset_index(drop=True, inplace=True)
df_exploded = df_exploded.dropna(subset=['content'])
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,“Cái làm ta hạnh phúc”,"[Rồi thêm chút công việc, Cho ta làm hàng ngày...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,“Chiều vừa xốp trên tay”,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,1,“Chiều vừa xốp trên tay”,"[Ớt đỏ sao cứ đỏ, Táo chín cho thật vàng, Em đ...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
4,2,“Dưới giàn hoa thiên lý...”,"[Dưới giàn hoa thiên lý, Một mình anh đang ngồ...","Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
...,...,...,...,...,...
481,98,Anh biết,"[Đất hiến dâng tuổi trẻ, Trời buông thả tuổi g...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
482,99,Anh bốn mùa lập đông,"[Buồn, thì buồn thiệt đó, Em à, có biết không,...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
483,99,Anh bốn mùa lập đông,"[Vâng, phiến buồn còn đó, Em nào biết phải khô...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
484,99,Anh bốn mùa lập đông,"[Ừ, nỗi buồn vậy đó, Em đừng vội bâng khuâng, ...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [8]:
df_exploded['content'] = df_exploded['content'].apply(lambda x: '\n'.join(x))
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,“Cái làm ta hạnh phúc”,Rồi thêm chút công việc\nCho ta làm hàng ngày\...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,1,“Chiều vừa xốp trên tay”,Ớt đỏ sao cứ đỏ\nTáo chín cho thật vàng\nEm đẹ...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
4,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
...,...,...,...,...,...
481,98,Anh biết,Đất hiến dâng tuổi trẻ\nTrời buông thả tuổi gi...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
482,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
483,99,Anh bốn mùa lập đông,"Vâng, phiến buồn còn đó\nEm nào biết phải khôn...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
484,99,Anh bốn mùa lập đông,"Ừ, nỗi buồn vậy đó\nEm đừng vội bâng khuâng\nV...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [12]:
from datasets import Dataset

poem_dataset = Dataset.from_pandas(df_exploded[['content']])
poem_dataset

Dataset({
    features: ['content', '__index_level_0__'],
    num_rows: 441
})

In [14]:
TEST_SIZE = 0.1
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['content', '__index_level_0__'],
        num_rows: 396
    })
    test: Dataset({
        features: ['content', '__index_level_0__'],
        num_rows: 45
    })
})

### 3. Preprocess dataset

In [15]:
MODEL_NAME = 'danghuy1999/gpt2-viwiki'

tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

/home/trongnhat/anaconda3/envs/poem/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
tokenizer.pad_token = tokenizer.eos_token
MAX_SEQ_LEN = 100

def preprocess_function(row):
    return tokenizer(
        row['content'],
        max_length=MAX_SEQ_LEN,
        padding='max_length',
        truncation=True
    )

tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=poem_dataset['train'].column_names,
)

Map (num_proc=4): 100%|██████████| 45/45 [00:00<00:00, 373.09 examples/s]


In [17]:
tokenizer('xin chào các bạn', max_length=10, padding='max_length', truncation=True)

{'input_ids': [6504, 3939, 331, 1486, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]}

In [18]:
tokenized_poem_dataset['train'][0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [19]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

### 4. Training

In [20]:
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)

In [22]:
training_args = TrainingArguments(
    output_dir='gpt2_viet_poem_generation',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    # fp16=True,
    push_to_hub=True
)

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset['train'],
    eval_dataset=tokenized_poem_dataset['test'],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

100%|██████████| 500/500 [21:42<00:00,  2.34s/it]

{'loss': 5.7855, 'learning_rate': 0.0, 'epoch': 10.0}


100%|██████████| 500/500 [21:44<00:00,  2.61s/it]


{'train_runtime': 1304.7933, 'train_samples_per_second': 3.035, 'train_steps_per_second': 0.383, 'train_loss': 5.78548974609375, 'epoch': 10.0}


TrainOutput(global_step=500, training_loss=5.78548974609375, metrics={'train_runtime': 1304.7933, 'train_samples_per_second': 3.035, 'train_steps_per_second': 0.383, 'train_loss': 5.78548974609375, 'epoch': 10.0})

In [29]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/Trongnhat191/gpt2_viet_poem_generation/commit/c4f33ee3c6aca686691365ed1b15f2ce8dda343c', commit_message='Training complete', commit_description='', oid='c4f33ee3c6aca686691365ed1b15f2ce8dda343c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Trongnhat191/gpt2_viet_poem_generation', endpoint='https://huggingface.co', repo_type='model', repo_id='Trongnhat191/gpt2_viet_poem_generation'), pr_revision=None, pr_num=None)

### 5. Inference

In [30]:
from transformers import pipeline

prompt = 'Con sông quê tôi đẹp\n'
generator = pipeline('text-generation', model='Trongnhat191/gpt2_viet_poem_generation')
results = generator(
    prompt,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
results = results[0]['generated_text']
print()
for line in results.split('\n'):
    print(line)

/home/trongnhat/anaconda3/envs/poem/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Con sông quê tôi đẹp
Mà qua đời anh ấy
Chém sâu vào con đường không lay
Nơi đời em, người còn sống.
Như vậy, vì vậy là ngày nay
Một số sách có ghi chép:
- "Trong hồ, có anh,


In [31]:
prompt = 'Học học nữa học mãi\n'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
outputs = model.generate(
    inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results = results[0]
print()
for line in results.split('\n'):
    print(line)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Học học nữa học mãi
Một sự thật lớn
Chầu đã thấy đầu!
Có một ý nghĩa, đang đứng
Đôi khi đó chỉ là?
Không cần học có được?
Long vật từ xa bao yên?
Nhưng người ta còn sống lại?

